In [0]:
%run ../../_utils

In [0]:
from pyspark.sql.functions import hour, minute, second, concat, lit, date_format, dayofweek, when, month, year
from pyspark.sql.types import StringType


# Camada Silver

Na camada silver, limpezas e ajustes em dados devem ser aplicados

Caso seja possível, enriquecer os dados e extrair dados também deve acontecer nessa camada

In [0]:
tb_name = "silver.supermarket_sales"
target_location = "dbfs:/delta/silver/supermarket_sales"

## 1 - Data ingestion

In [0]:
df = spark.read.table("bronze.supermarket_sales") # leituira da delta table

In [0]:
df.printSchema()

In [0]:
display(df.take(10))


## 2 - Data Munging

Processo de limpeza e normalizações necessárias

Como esses dados já são bem tratados, vamos apenas corrigir o campo "time" para conter a data e verificar a possibilidade de criação de algum outro campo (como campo timestamp ao unir date e time)


### timestamp da venda

Juntar a data/hora do campo "time" e unir com o campo "date"

In [0]:

# Extrair a hora, minutos e segundos
df = df.withColumns({"hora": hour(col("time")), 
                     "minuto": minute(col("time")),
                     "segundo": second(col("time"))}) # withColumns eu nao costumo usar, mas vi no linkedin e vim testar rs

# concatenação da data com a hora, minutos e segundos, além dos literais T, : e Z para formar string de timestamp
df = df.withColumn("sale_time", 
                   concat(
                       col("date").cast(StringType()), lit("T"),
                       col("hora").cast(StringType()), lit(":"),
                       col("minuto").cast(StringType()), lit(":"),
                       col("segundo").cast(StringType()), lit("Z")
                   ).cast("timestamp")) # cast para timestamp

# drop de campos intermediarios
df = df.drop("time","hora", "minuto", "segundo")


### mes/ano da venda

Campo desse tipo ajuda para calcular vendas mensais

In [0]:
df = df.withColumns(
    {
        "month_year": date_format("date", "MM-yyy"),
        "sale_month": month("date"),
        "sale_year": year("date"),
    }
)


### taxa por unidade

um campo apenas para manipulação de dados,
mas será apenas o total de taxa (5%) divididos pela quantidade vendida. tendo assim a taxa unitaria

In [0]:
df = df.withColumn("total_tax_unity", col("tax_5")/col("quantity"))

### Dia da semana (numero e descrição)

In [0]:
df = df.withColumn("number_day_of_week", dayofweek(col("date")))

# Traduzir os números do dia da semana para o português
df = df.withColumn(
    "day_of_week",
    when(df["number_day_of_week"] == 1, "Domingo")
    .when(df["number_day_of_week"] == 2, "Segunda-feira")
    .when(df["number_day_of_week"] == 3, "Terça-feira")
    .when(df["number_day_of_week"] == 4, "Quarta-feira")
    .when(df["number_day_of_week"] == 5, "Quinta-feira")
    .when(df["number_day_of_week"] == 6, "Sexta-feira")
    .when(df["number_day_of_week"] == 7, "Sábado"),
)


### weekend?

criar um campo para informar se a data é um fim de semana (não faz muito sentido, mas vamos supor que foi uma regra de negócio)

In [0]:
df = df.withColumn(
    "weekend",
    when(col("day_of_week").isin(["Sábado", "Domingo"]), True).otherwise(False),
)

In [0]:
display(df.take(10))


belezura!

Limpamos um cadim, enriquecemos e transformamos algumas coisas e temos nosso dataset preparado para analises (até preditiva e prescritiva -> mas aí tem que fazer um pouco mais de cleanings e etc)


## Saving data

In [0]:
tb_name = 'silver.supermarket_sales'
target_location = '/FileStore/delta/supermarket_sales/silver'

In [0]:
save_dataframe(df, "delta", target_location=target_location)


## create delta table

TODO: implementar UPSERT

o upsert serve para não precisar reescrever todos os dados, mas aproveitar do Delta para fazer um MERGE, caso um registro antigo tenha uma nova versão e INSERT para os dados que são novos

In [0]:
create_table(tb_name, target_location)

In [0]:
# exit para fechar a execução
dbutils.notebook.exit("OK")

In [0]:
%sql
select
  *
from
  silver.supermarket_sales
limit
  10